In [66]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import requests as req
from selenium.webdriver.common.action_chains import ActionChains
import csv
import random

In [67]:
#셀레니움 세팅
from selenium.webdriver.support.ui import WebDriverWait
#예외상황이 떴을 때 대처하는 프로그램
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

### 네이버 로그인
자동화된 크롬창을 실행시켜서 네이버 로그인 페이지 접속
    1. 크롬드라이버 제어
    2. 네이버 로그인(아이디 입력 -> 비밀번호 입력 -> 로그인 버튼 클릭 순서로 작업 진행)

In [68]:
#크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')
# 현재 컴큐터 크롬 드라이버 위치
chorme_path = './chromedriver.exe' #본인의 크롬 드라이버 위치 입력

# 자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)

C:\Users\user\AppData\Local\Temp\ipykernel_13420\2603244799.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)


In [69]:
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

In [70]:
# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.maximize_window()
driver.implicitly_wait(10)

# 본인의 아이디 , 비밀본호를 각 변수에 저장
my_id = 'id'
my_pw = 'pw'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

#'로그인' 버튼 클릭
driver.find_element('id','log.login').click()
time.sleep(1)

### 크롤링 할 카페 접속 및 원하는 게시판 클릭
1. 카페접속
- 로그인한 계정이 카페에 회원가입/등업조건 만족해야 합니다.
2. 원하는 게시판 선택(클릭)
- F12(개발자 도구) 눌러 크롤링 대상 게시판의 선택자를 확인


In [71]:
#카페 접속
time.sleep(2)
cafe_url = 'https://cafe.naver.com/fashionsale'
driver.get(cafe_url)
time.sleep(1)

#원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink37"]').click()
time.sleep(1)

#프레임 전환 iframe
driver.switch_to.frame('cafe_main')
time.sleep(1)

### 게시물 크롤링
1. 첫번째 게시물 클릭 - xpath 활용
2. 크롤링할 개수 지정
3. 크롤링

In [72]:
# 제목, 본문, 댓글, 시간 빈 리스트 생성
titles = [] #제목
reviews = [] #리뷰
comments = [] #댓글
date = [] #시간

count_view = 0 

# 1.첫번째 게시물 클릭(선택자 입력 : XPath)
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]').click()

# 2.크롤링(for문으로 지정한 게시물 개수까지 돌림)
for i in range(2) : #새로 창 열기
    for i in range(30) : #한 번에 몇 개의 게시물을 크롤링할 것인가
        driver.implicitly_wait(30)
        
        #제목 수집
        time.sleep(random.uniform(0.5,1))
        title = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)
        #본문 수집(3가지 경우 존재)
        try:
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text').text
            reviews.append(content)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR,'div.ContentRenderer').text
            except:
                content = [] #본문이 사진으로만 구성된 경우
                reviews.append(content)
        #댓글 수집(댓글이 없는 경우, 댓글이 있는 경우 2가지 존재)
        soup = bs(driver.page_source, 'lxml') #현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span', class_='text_comment')
        if len(iscomment) == 0 : #댓글이 없는 경우
            box = []
        else : #댓글이 있는 경우
            WebDriverWait(driver,15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')) )
                #웹 드라이버를 통해 브라우제서 최대 15초 기다려주고, text_comment(댓글 선택자)
            
            soup = bs(driver.page_source, 'lxml')
            iscomment = soup.find_all('span', class_='text_comment')
            box = []
            for i in iscomment:
                box.append([i.get_text()])
        comments.append(box)
        
        #날짜 수집
        times = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date.append(times)
        
        #count_view 크롤링 진행상황 (50개마다 출력)
        count_view = count_view + 1
        if count_view % 50 == 0 : #50으로 나누었을때 0이 되면 게시물 크롤링 완료라고 뜨도록 설정
            print('{}개 게시물 크롤링 완료'.format(count_view))
            
        else:
            pass
        
        #다음 글 클릭(다음글 버튼 클릭해서 연속적으로 크롤링)
        try:
            driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default').click()
            driver.implicitly_wait(25)
        except: #다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            current_url = driver.current_url #다음글 버튼이 사라져서 크롤링이 멈춘 url 가지고 오기
            driver.close()
            
            #크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)

            ##재로그인
            #네이버 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(9)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)

            #'로그인'버튼 클릭
            driver.find_element('id','log.login').click()
            time.sleep(1)
            
            #멈춘 부분의 링크 가지고 오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)

    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
    time.sleep(random.uniform(0.5,1))
    link = driver.find_element(By.XPATH,'//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")
    
    #현재 크롬창 닫기
    driver.close()
    
    #크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)
    
    ##재로그인
    #네이버 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(9)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    #'로그인'버튼 클릭
    driver.find_element('id','log.login').click()
    time.sleep(1)
    
    #다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')
    
#크롬 창 닫기
driver.quit()

        
   


C:\Users\user\AppData\Local\Temp\ipykernel_13420\3338433949.py:97: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)


50개 게시물 크롤링 완료


In [76]:
print(len(titles))

60


In [77]:
print(len(reviews))

60


In [78]:
print(len(comments))

60


In [79]:
#데이터 합치기 - DataFrame화
data = {'times': date, 'title': titles, 'review': reviews, 'comments':comments}
dataDF = pd.DataFrame(data)

In [80]:
dataDF


,times,title,review,comments
0,2022.09.13. 09:53,연휴 끝,뭐죠? 연휴 왜 끝난거죠 ? ㅠ\n푹 쉬니까 현생 돌아오기 더 힘드네여..\n다들 ...,"[[혐생 ..... ㅠㅠ], [이번 연휴만 기다리고 있습니다 ㅠ], [얼른 토요일이..."
1,2022.07.28. 09:52,오늘 저녁은 또 뭐먹을까여 ㅠ,맨날 밥 뭐먹을지 고민하는 것두 일 ㅠㅠ\n점심도 안먹었는데 저녁 고민즁 ㅋㅋㅋㅋㅋㅋㅋ,[[그것 참 고민이죠... 이럴때면 그냥 메뉴 정해놓고 도시락이나 구내식당이 있으면...
2,2022.06.28. 17:32,GS25 신발튀김 정품 문의 보셨나요?? (feat. 번장),GS25에서 신발 모양의 튀김을 만들었는데 정품문의 폭주하고 있대요ㅎㅎ\n번개장터에...,"[[ㅋㅋㅋㅋ와 신선함 인정], [ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ], [머지...?], [ㅋ⫬..."
3,2022.06.02. 10:43,2022 서울 재즈 페스티벌 약간 늦은 후기,안녕하세요! 여러분의 오덩이입니다. 지난 주 토요일에 처음으로 서울 재즈 페스티벌을...,"[[혼자갔어요? 개간지네], [저는… 아싸니까…], [아싸라기보단...돌싸 아닙니까..."
4,2022.04.07. 17:58,벚꽃이 활짝 폈네요~,오랜만에 벗꽃 나들이 가고 싶은데\n주말에 시간 내서 가볼까 해요 ㅎㅎ\n다들 올 ...,"[[서울은 벌써 피고 지더라구요], [벚꽃 너무 빨리졌어요 ㅜㅜ]]"
5,2022.03.30. 14:21,저녁으로 갈비 땡겨요 ㅎㅎ,다들 오늘 맛점 하셨나요?ㅎㅎ\n점심을 너무 적게 먹었더니 배가 고프네요\n오늘 저...,"[[갈비 맛있겠네용..], [맛있어요~! ㅎㅎ]]"
6,2022.03.28. 11:46,오늘은 놀러 가고 싶은 날이에요 ㅎㅎ,오늘 날씨가 진짜 좋네요\n아침엔 아직 춥지만 낮되면 온도 올라가서\n봄 나들이 하...,"[[그러니까요ㅎㅎ 얼른 꽃피면 좋겠네요], [맞아요 ㅎㅎ], [오늘 날씨 따뜻하네용..."
7,2022.03.24. 17:25,아직 봄은 아닌가 봐요 ㅋㅋ,매일 날씨 찾아봐야 하구\n이제 봄 인데도 \n옷깁기 진짜 힘든 시기네요 ㅎㅎ,"[[날씨 애매해요 요새 ㅠㅠ], [오늘은 날씨가 꽤 좋네요 ㅎㅎ], [맞아요 일교차..."
8,2022.03.16. 19:33,코로나 확진후 복귀했습니다..ㅎ,3월7일 자가키트에서 양성을 보여줘서 바로 보건소에서 pcr 검사를 받았는데\n역시...,"[[고생하셨습니다!!], [수고하셨어요~~], [고생하셨습니다 ㅎ], [고생하셨어..."
9,2022.03.02. 23:39,카페 등급이 자꾸 떨어지기만 해서...눙물이..,제곧내..후...,"[[ㅜㅜㅜㅠㅠㅠ다들 예전 같디 않네요..], [뿌엥...ㅠㅠ], [ㅠㅜ], [ㅠㅠ ..."


In [82]:
#csv로 저장
dataDF.to_csv('패션사관학교.csv', encoding='utf-8-sig') #저장할 파일명.csv 입력 후 셀 실행